In [1]:
from typing import List, Optional
from pathlib import Path
from collections import defaultdict

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataframes = sorted(Path('../results').glob('*.csv'))

In [4]:
key_columns = ['제목', '방송 시작']
columns = ['제목', '방송 기간', '방송 횟수', '장르', '채널', '제작사', '극본', '출연자', '방송 시작']

In [5]:
frames = {
    data.stem: pd.read_csv(str(data))
    for data in dataframes
}

for frame in frames.values():
    assert (frame.columns == columns).all()

In [6]:
total = defaultdict(set)

In [7]:
for key, frame in frames.items():
    for _, args in frame[key_columns].iterrows():
        total[tuple(args.values)].add(key)

## Merge multiple domain dataframes

In [8]:
def merge(values: List[str]) -> str:
    """Simple merge return maximum length data
    """
    def valuable(value: np.ndarray) -> int:
        item = value.item()
        if isinstance(item, str):
            return len(item)
        return 0
    
    index = np.array(tuple(map(valuable, values))).argmax()
    
    return values[index].item()

In [9]:
table = pd.DataFrame([], columns=columns)

In [10]:
for key, domains in total.items():
    if len(domains) > 1:
        table = table.append(pd.Series([merge([frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)][column].values for domain in domains]) for column in columns], index=columns), ignore_index=True)
    else:
        domain, *_ = domains
        table = table.append(frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)])

In [11]:
print(table.shape)
table.head()

(1917, 9)


,제목,방송 기간,방송 횟수,장르,채널,제작사,극본,출연자,방송 시작
0,외과의사봉달희,2007년 1월 17일 ~ 2007년 3월 15일,18부작,의학 드라마,SBS TV,DSP 미디어,[[이정선 (작가),"이요원, 이범수, 김민준, 오윤아 외",2007-01-17 00:00:00
1,"단,하나의사랑",2019년 5월 22일 ~ 2019년 7월 11일,32부작,"판타지, 로맨스",KBS 2TV,"조윤정, 정해룡\n 빅토리콘텐츠, 몬스터유니온",최윤교,"신혜선, 엘 (가수), 이동건, 김보미 (1987년) 외",2019-05-22 00:00:00
2,백일의낭군님,2018년 9월 10일 ~ 2018년 10월 30일,16부작,"미스터리 로맨스, 픽션사극",tvN,에이스토리\n 이상백,노지설,"도경수, 남지현 (배우), 조성하 (배우) 외",2018-09-10 00:00:00
3,러브홀릭,2005년 5월 2일 ~ 2005년 6월 21일,16부작,"멜로, 드라마",KBS 2TV,KBS 2TV 자체제작,[[이향희 (작가),"강타, 김규리 (1979년 8월), 이선균, 유인영 외",2005-05-02 00:00:00
4,왜그래풍상씨,2019년 1월 9일 ~ 2019년 3월 14일,40부작]] + 스페셜 1부작,"가족, 코미디",KBS 2TV,"김상헌, 조형진\n 초록뱀미디어, 팬 엔터테인먼트",문영남,"유준상 (배우), 오지호 (배우), 전혜빈, 이시영 (배우), 이창엽 (배우) 외",2019-01-09 00:00:00


In [12]:
table.to_csv('../results/results.csv', index=None)